# 1. Academia

## 1.1. 데이터 병합

In [48]:
import pandas as pd
from pandas import value_counts

# 파일 경로
files = [
    "DSS/dss_vol164to170.csv",
    "DSS/dss_vol170to180.csv",
    "DSS/dss_vol180to190.csv",
    "DSS/dss_vol183to190.csv",
    "DSS/dss_vol186to190.csv",
    "DSS/dss_vol189to197.csv"
]

# 모든 파일 읽어서 합치기
dfs = [pd.read_csv(f) for f in files]
merged_df = pd.concat(dfs, ignore_index=True)

# 중복 제거 (같은 논문 여러 번 수집된 경우)
merged_df = merged_df.drop_duplicates()

# CSV로 저장
merged_df.to_csv("dss_merged.csv", index=False)

print("CSV 병합 완료: dss_merged.csv")

CSV 병합 완료: dss_merged.csv


In [50]:
merged_df.to_csv('DSS/DSS.csv', index=False, encoding="utf-8-sig")

In [52]:
import pandas as pd
import glob
import os

# CSV 파일이 들어있는 폴더 경로
folder_path = "Crawler/IAM"

# 해당 폴더 안의 모든 csv 파일 불러오기
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 파일들을 DataFrame으로 읽어서 합치기
dfs = [pd.read_csv(f) for f in all_files]
merged_df = pd.concat(dfs, ignore_index=True)

# 중복 제거 (같은 논문 여러 번 수집된 경우)
merged_df = merged_df.drop_duplicates()

# 저장
output_file = os.path.join(folder_path, "IAM.csv")
merged_df.to_csv(output_file, index=False)

print("CSV 병합 완료:", output_file)

CSV 병합 완료: IAM/IAM.csv


In [53]:
import pandas as pd
import glob
import os

# CSV 폴더 경로
folder_path = "Crawler/Academia"

# 모든 csv 파일 읽기
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 저널 키워드별 분류
journals = {
    "isre": [f for f in all_files if "isre" in os.path.basename(f).lower()],
    "misq": [f for f in all_files if "misq" in os.path.basename(f).lower()],
    "dss":  [f for f in all_files if "dss"  in os.path.basename(f).lower()],
    "iam":  [f for f in all_files if "iam"  in os.path.basename(f).lower()],
}

# 저널별 병합
for journal, files in journals.items():
    if not files:
        continue
    dfs = [pd.read_csv(f) for f in files]
    merged = pd.concat(dfs, ignore_index=True).drop_duplicates()
    output_file = os.path.join(folder_path, f"{journal.upper()}.csv")
    merged.to_csv(output_file, index=False)
    print(f"{journal.upper()} 병합 완료: {output_file}")

ISRE 병합 완료: Academia/ISRE.csv
MISQ 병합 완료: Academia/MISQ.csv
DSS 병합 완료: Academia/DSS.csv
IAM 병합 완료: Academia/IAM.csv


## 1.2. 데이터 전처리

In [18]:
import pandas as pd
import glob
import os

# 1. 폴더 및 파일 경로 설정
folder_path = 'Data/Academia'  # CSV 파일들이 있는 폴더
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

df_list = []

for file in csv_files:
    # 파일명에서 저널명 추출 (예: 'MISQ.csv' → 'MISQ')
    journal_name = os.path.splitext(os.path.basename(file))[0].upper()

    # 파일 불러오기
    df = pd.read_csv(file)

    # journal 컬럼은 affiliations 대체용으로만 사용 (최종 저장에는 포함 X)
    df['journal'] = journal_name

    # affiliations 컬럼이 없으면 새로 생성
    if 'affiliations' not in df.columns:
        df['affiliations'] = journal_name
    else:
        df['affiliations'] = df['affiliations'].fillna(journal_name)

    # year 추출 (내부 전처리용)
    if journal_name == 'MISQ':
        def extract_year_from_doi(doi):
            if pd.isna(doi): return None
            doi_lower = doi.lower()
            if 'vol47' in doi_lower: return 2023
            if 'vol48' in doi_lower: return 2024
            if 'vol49' in doi_lower: return 2025
            return None
        df['date'] = df['url'].apply(extract_year_from_doi)
    else:
        df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.year

    df_list.append(df)

# 2. 모든 데이터 통합
merged_df = pd.concat(df_list, ignore_index=True)

# 3. 중복 제거 (title + abstract 기준)
merged_df.drop_duplicates(subset=['title', 'abstract'], inplace=True)

# 4. Null 제거 (title, abstract)
merged_df.dropna(subset=['title', 'abstract'], inplace=True)

# 5. 최종 컬럼 순서 지정 (keyword 또는 affiliations이 없는 경우 대비)
final_cols = ['title', 'abstract', 'keywords', 'date', 'affiliations']
for col in final_cols:
    if col not in merged_df.columns:
        merged_df[col] = None  # 없으면 빈 컬럼 생성

final_df = merged_df[final_cols]  # 순서 맞춰서 정렬

# 6. 저장
final_df.to_csv('Data/Academia/merged_journals_preprocessed.csv', index=False)

# 7. 확인용 출력
print("✅ 최종 데이터 수:", final_df.shape)
print("✅ 결측치 확인:\n", final_df.isnull().sum())

✅ 최종 데이터 수: (4032, 5)
✅ 결측치 확인:
 title            0
abstract         0
keywords        47
date             0
affiliations     0
dtype: int64


In [19]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4032 entries, 1 to 4373
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         4032 non-null   object
 1   abstract      4032 non-null   object
 2   keywords      3985 non-null   object
 3   date          4032 non-null   int64 
 4   affiliations  4032 non-null   object
dtypes: int64(1), object(4)
memory usage: 189.0+ KB


In [20]:
print(final_df.value_counts(subset=['affiliations'],sort=False).sort_index())

affiliations
DSS              360
HICSS           2097
IAM              270
ICIS             822
ISR              268
MISQ             215
Name: count, dtype: int64


In [15]:
import pandas as pd
df = pd.read_csv('Data/Academia/03_journal_2023_2025.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3209 entries, 0 to 3208
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         3209 non-null   object
 1   date          3209 non-null   int64 
 2   abstract      3209 non-null   object
 3   keywords      3209 non-null   object
 4   authors       3209 non-null   object
 5   affiliations  3209 non-null   object
dtypes: int64(1), object(5)
memory usage: 150.6+ KB


In [16]:
print(df.value_counts(subset=['affiliations'],sort=False).sort_index())

affiliations
HICSS           2098
ICIS             822
ISR              149
MISQ             140
Name: count, dtype: int64


# 2. Industry

## 2.1. 데이터 병합